In [ ]:
import os
import sys
import time
import json
import shutil
import codecs
import requests
import psycopg2
import feedparser
import sqlalchemy
import urllib.parse

import numpy as np
import pandas as pd

from random import random
from datetime import datetime
from datetime import timezone
from selenium.webdriver import Chrome

from utils.sql_utils import *
from utils.scrape_utils import *
from utils.data_science_utils import *

In [ ]:
host=sql_utils.Host.LOCAL_JEAN
schema=models.schema
verbose=Verbose.INFO

In [ ]:
loadRoutine(host=host, schema=schema, verbose=verbose)

In [ ]:
pipelineNYTHeadlines(fetchSource=True, host=host, schema=schema, verbose=verbose)

In [ ]:
pipelineNYTNewsWire(fetchSource=True, verbose=Verbose.INFO) # startPage = 0

In [ ]:
pipelineNewsAPIHeadline(fetchSource=True)

driver.get("https://www.nytimes.com/")

Possible sources:

    "NYTimes": "https://www.nytimes.com/",
    "BBC": "https://www.bbc.co.uk/",
    "CNN": "https://edition.cnn.com/",
    "FoxNews": "https://www.foxnews.com/",
    "OAN": "https://www.oann.com/"

The Guardian, Yahoo news, Washington Post, Daily Mail, ...

https://www.4imn.com/top200/ for more

In [ ]:
all_sources = pd.read_sql("select * from newsdb.sources", getEngine())
all_sources.shape[0]

In [ ]:
all_articles = pd.read_sql_table("articles", getEngine())
all_articles.shape[0]

In [ ]:
all_article_contents = pd.read_sql_table("article_contents", getEngine())
all_article_contents.shape[0]

In [ ]:
sections = getNYTSections(source.api_key)

In [ ]:
createNlpModel(attributes=['title', 'description'])

In [ ]:
getSimilarArticlesFromText('VP pick', 'title', 10)

In [ ]:
getSimilarArticlesFromText('VP pick', 'description', 10)

In [ ]:
getSimilarArticlesFromText('Lebanon minister resigns in wake of deadly Beirut blast', 'title', 10)

In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
url = "https://www.bbc.com/news/live/world-53775682"

In [ ]:
url = "https://www.20min.ch/fr/story/avec-ses-propos-sur-son-remede-trump-cree-la-confusion-634629791483"

In [ ]:
url = "https://nyti.ms/2GS6cZT"

In [ ]:
source = getSourceFromUrl(url)
source

In [ ]:
title = scrapeArticleTitle(url)
title

In [ ]:
q = getDBSession().query(
    models.Article.article_uuid, models.Article.title,
    models.Article.description, models.Article.article_url, models.Source.source_name
).filter(
    models.Article.description.isnot(None)
).join(
    models.Source, models.Article.source_uuid == models.Source.source_uuid
).limit(
    200
)

In [ ]:
df = pd.read_sql(q.statement, q.session.bind)

In [ ]:
getSourceFromUrl("https://www.espn.com/nfl/story/_/id/29526181/season-impact-future-salary-caps-central-nfl-talks-sources-say")

In [ ]:
getSourceFromUrl("https://www.houstonchronicle.com/texas-sports-nation/astros/article/Astros-Justin-Verlander-out-for-season-with-15435496.php")

In [ ]:
getSourceFromUrl('https://www.express.co.uk/news/science/1312473/Comet-NEOWISE-tonight-where-is-Comet-NEOWISE-now')

In [ ]:
getSourceFromUrl('https://www.cbsnews.com/news/mountain-rescue-crew-carry-saint-bernard-off-scafell-pike-england-2020-07-26/')

In [ ]:
getSourceFromUrl('https://finance.yahoo.com/news/gold-futures-touch-record-haven-221708713.html')

In [ ]:
df_sources = list(map(lambda x: getSourceFromUrl(x).source_name, list(df['article_url'])))

In [ ]:
for i in range(len(df_sources)):
    if df_sources[i] != df['source_name'][i]:
        print(i, df_sources[i], '!=', df['source_name'][i])

In [ ]:
npr_feed_url = "https://feeds.npr.org/%d/rss.xml"

In [ ]:
for i in range(200, 50000):
    data = feedparser.parse(npr_feed_url % i)
    if 'feed' in data and 'title' in data['feed']:
        print("(\"" + (npr_feed_url % i) + "\", \"" + data['feed']['title'].replace(" : NPR", '') + "\"),")

In [8]:
len(sql_utils.getDBSession().query(models.Source).filter(models.Source.language == 'fr').all())

6

In [3]:
import logging
from utils import scrape_utils

logger = logging.getLogger()
logger.setLevel(logging.INFO)

#scrape_utils.importNewsAPISources(language='fr')

In [7]:
scrape_utils.importNewsAPISources(language='fr')

INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=fr&apiKey=e30a64cfe1734e6794bdab67106590fa


0

In [4]:
languages = 'ar de en es fr he it nl no pt ru se ud zh'
countries = 'ae ar at au be bg br ca ch cn co cu cz de eg fr gb gr hk hu id ie il in it jp kr lt lv ma mx my ng nl no nz ph pl pt ro rs ru sa se sg si sk th tr tw ua us ve za'

In [1]:
from utils import sql_utils
from utils import models
import sqlalchemy
hist = sql_utils.getDBSession().query(models.Source.added_at, sqlalchemy.func.count(models.Source.source_uuid)).group_by(models.Source.added_at).order_by(models.Source.added_at).all()
#session.query(Table.column, func.count(Table.column)).group_by(Table.column).all()

In [2]:
hist[0][0]

datetime.datetime(2020, 7, 27, 8, 50, 5, 289967)

In [9]:
import time
count = 0
for l in languages.split():
    count += scrape_utils.importNewsAPISources(language=l)
    time.sleep(1)
print(count)

INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=ar&apiKey=e30a64cfe1734e6794bdab67106590fa
INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=de&apiKey=e30a64cfe1734e6794bdab67106590fa
INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=en&apiKey=e30a64cfe1734e6794bdab67106590fa
ERROR:root:No source found for article http://espn.go.com
INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=es&apiKey=e30a64cfe1734e6794bdab67106590fa
INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=fr&apiKey=e30a64cfe1734e6794bdab67106590fa
INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=he&apiKey=e30a64cfe1734e6794bdab67106590fa
INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=it&apiKey=e30a64cfe1734e6794bdab67106590fa
INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=nl&apiKey=e30a64cfe1734e6794bdab67106590fa
INFO:root:Fetching https://newsapi.org/v2/sources?page=1&language=no&a

In [11]:
sql_utils.getDBSession().query(sqlalchemy.func.count(models.Source.source_uuid), models.Source.language).group_by(models.Source.language).all()

[(10, 'de'),
 (1, 'nl'),
 (5, 'fr'),
 (1, 'multi'),
 (101, None),
 (2, 'se'),
 (7606, 'en'),
 (1, 'ud'),
 (4, 'ru'),
 (2, 'ar'),
 (1, 'he'),
 (2, 'no'),
 (3, 'pt'),
 (2, 'zh'),
 (3, 'it'),
 (6, 'es')]

In [32]:
languages.split()[4]

'fr'

In [33]:
language_sources = sql_utils.getDBSession().query(models.Source).filter(models.Source.language == languages.split()[4]).all()
len(language_sources)

5

In [44]:
language_source_i = language_sources[3]

In [45]:
language_articles = sql_utils.getDBSession().query(models.Article).filter(models.Article.source_uuid == str(language_source_i.source_uuid)).all()
len(language_articles)

1

processing language de
   processing article 2c650e11-2fbb-4378-8523-53c5b709b6b7
   processing article eaaf0e83-873c-4c80-8f6a-66d179a853e6
processing language fr
   processing article 74cc8c1b-8e1d-4ac1-9e71-2a6f6deaac02
   processing article a1192f77-e9d0-4059-844a-916189c66253
processing language multi
   processing article d1322089-c897-4ed5-8c42-18731c57a76c
   processing article eb52e273-712e-4b89-b1f2-d6b7cb9d64df
processing language None
   processing article 4d3db52c-d3bc-44f2-81f5-cb13c0a5a087
   processing article f5787777-d526-4e72-b49a-ac249d2b9f73
processing language zh
   processing article f5200631-31da-46a7-b178-24c89668a300
   processing article 07272a02-adfe-4e49-a257-54b8d3d20414
processing language en
processing language it
   processing article 98fe9b88-c5a1-41cb-9dab-74c336c73b7b
   processing article 2b4b9c60-7604-4e3f-bff9-e6ea5361ee2c
processing language es
   processing article 1fa47333-0325-46d9-8f1c-30a7aea483ea
   processing article e0e3da41-62a8-42ad-9fb

In [98]:
articles_by_language = sql_utils.getDBSession().query(sqlalchemy.func.array_agg(models.Article.article_uuid), models.Source.language).join(models.Source, models.Article.source_uuid == models.Source.source_uuid).group_by(models.Source.language).all()

In [99]:
len(articles_by_language)

10

In [100]:
language_i = 1

In [101]:
len(articles_by_language[language_i])

2

In [102]:
articles_by_language[language_i][1]

'fr'

In [76]:
# articles_by_language[language_i][1]

['Comment la baleine est devenue le plus gros des animaux',
 'Are the batteries of electric cars clean?']

In [77]:
articles_by_language[language_i][0]

[UUID('74cc8c1b-8e1d-4ac1-9e71-2a6f6deaac02'),
 UUID('a1192f77-e9d0-4059-844a-916189c66253')]

In [83]:
article_i = 0

In [84]:
articles_by_language[language_i][0][article_i]

UUID('74cc8c1b-8e1d-4ac1-9e71-2a6f6deaac02')

In [85]:
articles_by_language[language_i][1][article_i]

'Comment la baleine est devenue le plus gros des animaux'

In [86]:
from utils import translate_utils

In [96]:
article = sql_utils.getDBSession().query(models.Article).filter(models.Article.article_uuid==str(articles_by_language[language_i][0][article_i])).first()

In [106]:
title_translation = translate_utils.translateText(article.title)

In [107]:
title_translation.detected_language_code

'fr'

In [108]:
title_translation.translated_text

'How the whale became the biggest animal'

In [104]:
if article.description is not None:
    description_translation = translate_utils.translateText(article.description)

In [91]:
mutli_lingual_article = models.MultiLingualArticle(
    article_uuid=str(articles_by_language[language_i][0][article_i]),
    title=article.title,
    description=article.description,
    language=title_translation.detected_language_code
)

In [93]:
sql_utils.insertEntry(mutli_lingual_article)

True

In [97]:
article

<Article 74cc8c1b-8e1d-4ac1-9e71-2a6f6deaac02: Comment la baleine est devenue le plus gros des animaux>

In [109]:
article.title=title_translation.translated_text

In [110]:
sql_utils.commitSession()